In [1]:
%matplotlib widget
import glob
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable

from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.visualization import ImageNormalize, SqrtStretch, LogStretch, LinearStretch, ZScaleInterval, ManualInterval
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from matplotlib import ticker
# plt.style.use('dark_background')
plt.style.use('ggplot')

import numpy as np

In [2]:
data_path = '/Users/nmiles/hst_cosmic_rays/analyzing_cr_rejection/stis_test/long_clean'

In [3]:
def plot_image(data, norm=None, units=None, title=None, xlim=None, ylim=None):
    fig, ax = plt.subplots(nrows=1, ncols=1)
    im = ax.imshow(data, norm=norm, origin='lower', cmap='viridis')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(im, cax=cax)
    cbar.set_label(f"{units}")
    ax.grid(False)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(title)
    plt.show()

In [4]:
N=20
flist = glob.glob(data_path+'/*flt.fits')[:N]

In [5]:
hdu = fits.open(flist[0])
data = hdu[1].data
units = hdu[1].header['BUNIT']
exptime = hdu[0].header['TEXPTIME']
hdu.close()

In [6]:
norm = ImageNormalize(data, stretch=SqrtStretch(), vmin=0, vmax=50*np.median(data))
cbar_bounds = [i for i in range(0,500,70)]
sci_cmap = plt.cm.viridis
norm1 = colors.BoundaryNorm(boundaries=cbar_bounds,
                                  ncolors=sci_cmap.N)

In [7]:
global_mean, global_median, global_std = sigma_clipped_stats(data, sigma=5, maxiters=3)

In [8]:
print(f"mean: {global_mean:.3f}\nmedian: {global_median:.3f}\nstd: {global_std:.3f}")

mean: 40.249
median: 25.989
std: 58.571


In [9]:
plot_image(data, norm=norm, units=units, title=f"Exposure Time: {exptime:0.0f}s")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Visualization the CR rejection algorithm
- The following cells contain a series of functions and widgets that are combined to create an interactive visualization tool for analyzing the principles behind the CR algorithm

In [10]:
import ipywidgets as widgets
from ipywidgets import interact, fixed, interactive, VBox, HBox

Define some preset values and analyze the list of data to extract the pixel value for 

In [203]:
x0=434
y0=434
# pixval = []
# for f in flist:
#     data = fits.getdata(f)
#     pixval.append(data[y0][x0])

In [204]:
# @interact(fname=file_slider, x=fixed(512), y= fixed(512), norm=fixed(norm))
def interactive_plot_image(
    fname,
    norm,
    x=fixed(434),
    y=fixed(434),
    ax=None,
    fig=None,
    units=None,
    w=10,
    h=10
):
    
    ax.clear()
    texptime = fits.getval(fname, keyword='TEXPTIME')
    data = fits.getdata(fname)
    patch = patches.Rectangle(xy=(x-0.5, y-0.5), width=1, height=1, fill=False, edgecolor='r', lw=2.25)
    im = ax.imshow(data, norm=norm, origin='lower')
    ax.set_xlim((x-w, x+w))
    ax.set_ylim((y-h, y+h))
    ticks = [y-i for i in range(1,11)] + [y] + [y+j for j in range(1,11)]
    ticks.sort()
    ax.add_patch(patch)
    ax.grid(False)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="10%", pad=0.15)    
    cbar = fig.colorbar(im, cax=cax, orientation='vertical')
#     n = len(cbar.ax.get_yticklabels())
    tick_locator = ticker.MaxNLocator(6)
    cbar.ax.set_yticklabels(
        cbar.ax.get_yticklabels(),       
        rotation=-10,
        horizontalalignment='left',
        fontsize=8
    )
    cbar.locator = tick_locator
    cbar.update_ticks()
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.tick_params(axis='both', which='both', width=1.5)


    cbar.set_label(f"{units}", fontsize=10)
#     cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(), horizontalalignment='left', rotation=-25, fontsize=10)
#     ax.set_title(f"{os.path.basename(fname)}, {texptime:.0f}s")
    fig.suptitle(f"Current Image: {os.path.basename(fname)}, Exposure Time: {texptime:.0f}s")
    ax.set_xlabel('X [pixel]')
    ax.set_ylabel('Y [pixel]')
    
    

In [260]:
def plot_pix_vals(
    current_file,
    flist=None, 
    current_img_color='red',
    c='k',  
    x=512, 
    y=512, 
    ax=None,
    units=None,
    ymin=None,
    ymax=None,
):
    
    
    ax.clear()
    pixvals = []
    scatter_color = []
    scatter_marker = []
    labels = []
    for i, f in enumerate(flist):
        data = fits.getdata(f)
        pixval = data[y][x]
        pixvals.append(pixval)
        if f == current_file:
            current_im = ax.scatter(i+1, pixval, label=f"Current: {pixval:.2f}", c=current_img_color)
        else:
            ax.scatter(i+1, pixval, c=c)
        
   
#     scat = ax.scatter([i for i in range(1,len(pixval)+1)], pixval, label=labels, c=scatter_color )
#     ax.set_title(os.path.basename(current_file))
    ax.set_xlim((0,22))
    med = np.nanmedian(pixvals, axis=0)
    mean = np.nanmean(pixvals, axis=0)
    minim = np.nanmin(pixvals, axis=0)
    std = np.nanstd(pixvals, axis=0)
    ax_med = ax.axhline(med,ls='--',c='k', label=f"med: {med:.2f}")
    ax_mean = ax.axhline(mean, ls='-', c='k', label=f"mean: {mean:.2f}")
    ax_min = ax.axhline(minim, ls=':', c='k', label=f"min: {minim:.2f}")
    if ymin is None or ymax is None:
        ax.set_ylim((0, ymax+2*std))
    else:
        ax.set_ylim((ymin, ymax))
    ax.set_ylabel(units)
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.tick_params(axis='both', which='both', width=1.5)
 
    ax_legend = ax.legend(handles=[current_im, ax_med, ax_mean, ax_min],
                          loc='upper right', edgecolor='k')
    
    return ax

Setup a slider to control the file we are examining

In [261]:
file_slider1 = widgets.Select(
    options=flist,
    value=flist[np.argmax(pixval)],
    description='Image Plot',
    continuous_update=True,
    orientation='horizontal',
    readout=True, 
)
file_slider2 = widgets.Select(
    options=flist,
    value=flist[np.argmax(pixval)],
    description='Scatter Plot',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)
xslider = widgets.IntText(
    options=[i for i in range(1,1025)],
    value=x0,
    description='X Coordinate',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)
yslider = widgets.IntText(
    options=[i for i in range(1,1025)],
    value=y0,
    description='Y Coordinate',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)
wslider = widgets.IntSlider(
    min=5,
    max=200,
    step=5,
    value=20,
    description='Width',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)
hslider = widgets.IntSlider(
    min=5,
    max=200,
    step=5,
    value=20,
    description='Height',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)
ymin_slider = widgets.IntText(
    options=[i for i in range(1,1025)],
    value=0,
    description='ymin',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)
ymax_slider = widgets.IntText(
    options=[i for i in range(1,1025)],
    value= global_mean + 5*global_std,
    description='ymax',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True 
)

In [262]:
l = widgets.link((file_slider1, 'value'), (file_slider2, 'value'))

In [263]:
out = widgets.Output(layout={'border': '1px solid black'})


In [267]:
with out:
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, gridspec_kw={'wspace':0.75, 'hspace':0.5})
    w1 = interactive(
        interactive_plot_image, 
        fname=file_slider1,fig=fixed(fig),
        x=xslider, y=yslider,w=wslider,h=hslider,
        norm=fixed(norm), ax=fixed(ax1), 
        units=fixed('COUNTS')
    )
    w2 = interactive(
        plot_pix_vals, 
        current_file=file_slider2,
        x=xslider, y=yslider, w=wslider, h=hslider, ymin=ymin_slider, ymax=ymax_slider,
        flist=fixed(flist), ax=fixed(ax2),
        units=fixed('COUNTS'), current_img_color=fixed('r'), c=fixed('k')
    )
    hbox = HBox([w1, w2])
    w = widgets.GridBox([w1, w2], layout=widgets.Layout(grid_template_columns="repeat(2,50%)"))
    display(w)

In [265]:
plt.close('all')
out.clear_output()

In [266]:
out

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'display_data', 'data': {'text/plain'…